# A Simple Convolution Network

- Sobel operator in $x$ direction = 
$
    \begin{bmatrix} 
        -1 & 0 & 1 \\
        -2 & 0 & 2 \\
        -1 & 0 & 1
    \end{bmatrix}
$

- One (input, output) image tuple is given.
- http://docs.opencv.org/3.2.0/d2/d2c/tutorial_sobel_derivatives.html

In [44]:
import os
print (os.getcwd())
import numpy as np
import tensorflow as tf
import cv2 # opencv
from matplotlib import pyplot as plt


/Users/yndk/Desktop/github/tensorflow-learning


In [22]:
def readimagefiles(filelistname):
    # lines = open(filelistname, 'r').readlines()
    # print (lines)
    i = 0
    ins = []
    outs = []
    in_img = None
    with open(filelistname, 'r') as file:
        for line in file:
            if line[0]!='#':
                i += 1
                print (i, line)
                words = line.split()
                print (words)
                in_img = cv2.imread(words[0],0)
                #print ('# in_img = ', in_img)
                ins.append(in_img)
    return in_img # only one image is needed now.

def do_sobel (gray):
    laplacian = cv2.Laplacian(gray,cv2.CV_64F)
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=3)
    return np.float32(sobelx)

In [31]:
image = readimagefiles('sobel.txt')
#print ('image=', image, image.shape, image.dtype)
sobelx = do_sobel(image)
#print ('sobelx=', sobelx, sobelx.shape, sobelx.dtype)

image = np.float32(image)/255.;
sobelx = np.float32(sobelx)/255.;

print ('image=', image, image.shape, image.dtype)
print ('sobelx=', sobelx, sobelx.shape, sobelx.dtype)

1 sobel/autumn.jpg

['sobel/autumn.jpg']
image= [[ 0.88235295  0.87843138  0.81176472 ...,  0.29411766  0.29019609
   0.29411766]
 [ 0.69411767  0.81176472  0.82745099 ...,  0.26274511  0.27058825
   0.26274511]
 [ 0.28627452  0.48627451  0.627451   ...,  0.24705882  0.23921569
   0.22745098]
 ..., 
 [ 0.38039216  0.38039216  0.39607844 ...,  0.08627451  0.10588235
   0.11372549]
 [ 0.31764707  0.35686275  0.42352942 ...,  0.09803922  0.10588235
   0.08235294]
 [ 0.39215687  0.40000001  0.42745098 ...,  0.20392157  0.18039216
   0.1254902 ]] (240, 320) float32
sobelx= [[ 0.          0.1254902   0.05490196 ...,  0.07058824  0.          0.        ]
 [ 0.          0.53725493  0.18431373 ...,  0.03921569 -0.01960784  0.        ]
 [ 0.          0.45490196  0.45490196 ..., -0.04705882 -0.04705882  0.        ]
 ..., 
 [ 0.          0.16862746  0.16862746 ...,  0.          0.03529412  0.        ]
 [ 0.          0.26274511  0.14117648 ..., -0.03137255 -0.08235294  0.        ]
 [ 0.          0.2

### Now, the goal of the project is to make a convnet produce sobelx given image as an input.


In [42]:
X = tf.placeholder (tf.float32, shape=[None]+list(image.shape)+[1])
Y = tf.placeholder (tf.float32, shape=[None]+list(sobelx.shape)+[1])
feed_dict = {X: image, Y:sobelx}

print ('X=', X)
print ('Y=', Y)

# define NN model
filter2d = tf.Variable(tf.random_normal([3,3,1,1], stddev=0.01))
stride = 1
strides = [1,stride,stride,1]
padding = 'SAME'
conv2d = tf.nn.conv2d (X, filter2d, strides, padding, use_cudnn_on_gpu=True)

print ('filter2d=', filter2d)
print ('conv2d=', conv2d)

model = conv2d # goal: output of conv2d == sobel 

X= Tensor("Placeholder_15:0", shape=(?, 240, 320, 1), dtype=float32)
Y= Tensor("Placeholder_16:0", shape=(?, 240, 320, 1), dtype=float32)
filter2d= Tensor("Variable_6/read:0", shape=(3, 3, 1, 1), dtype=float32)
conv2d= Tensor("Conv2D_4:0", shape=(?, 240, 320, 1), dtype=float32)


In [43]:
cost = tf.reduce_mean (tf.square(model - Y))
trainer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
with tf.Session() as ss:
        ss.run(tf.global_variables_initializer())
        for 